## Imports...

In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
from collections import defaultdict

### First load all of the static image gestures. These are gestures that have no movement.

In [3]:
gesture_path = './gestures'
# If you recieve nothing from these lines make sure you have run nthe data_in scripts.
gesture_files = glob.glob("./gestures/*.csv")

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


def remove_suffix(text, suffix):
    if text.endswith(suffix):
        return text[: -1 * len(suffix)]
    return text


static_gestures = {}
# iterate through all of the gestures... Static pose is finding the median point of the dataframe.

# TODO: refit the transcription process to lead from correct pose point to pose point. instead of 0-20, 
# we need to do the math to the connecting limb to the center of the heart.
for indx, gesture_file in enumerate(gesture_files):
  
  df = pd.read_csv(gesture_file)
  
  gesture_file = remove_prefix(gesture_file, './gestures/')
  gesture_file = remove_suffix(gesture_file, '.csv')
  
  file_parts = gesture_file.split('_')
  gesture_name = file_parts[0].lower()
  gesture_part = f"{file_parts[2]}".lower()
  if gesture_name == 'one':
    print(gesture_part)

  if df.empty:
      continue
  
  # convert string to time.
  df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S.%f').dt.time
  
  sdf = np.sort(df['time'].to_numpy(), axis=0)
  # Get the lower level bound median
  ts_median = sdf[math.floor(len(sdf) / 2 )]
  
  # Get the static point image/gesture...
  ndf = df[df['time'].isin([ts_median])]
  newGestureData = static_gestures.get(gesture_name, {})
  newGestureData[gesture_part] = ndf
  static_gestures[gesture_name] = newGestureData
  
print(static_gestures['one'])

right
left
pose
{'right':                 time  index     x_angle     y_angle     z_angle         x  \
294  00:00:02.048922      0  271.417393   15.672960  344.322478  0.403672   
295  00:00:02.048922      1  270.946539  357.041610    2.958793  0.406900   
296  00:00:02.048922      2  272.496738  336.309495   23.710533  0.421388   
297  00:00:02.048922      3  309.456157  276.189247   85.213704  0.437194   
298  00:00:02.048922      4  151.608977   33.411891  234.216412  0.426257   
299  00:00:02.048922      5  268.846110  354.233684    5.767478  0.433733   
300  00:00:02.048922      6  271.545680  352.484799    7.517905  0.438970   
301  00:00:02.048922      7  270.495792  352.449447    7.550832  0.443120   
302  00:00:02.048922      8   90.178932  357.842362  177.842352  0.449079   
303  00:00:02.048922      9  268.646407    0.327533  359.672376  0.448800   
304  00:00:02.048922     10   72.341414   29.237257  210.430534  0.439842   
305  00:00:02.048922     11   79.233105   16.18239

In [5]:
# Loop through our images to a static pose we know works.

all_gestures = static_gestures.keys()
  
# Proof of concept for the hand gesture "1"
test_pose_name = 'one'
static_base_gesture = static_gestures['one']['right'].to_numpy()

for pose_name in all_gestures:
  # print(pose_name)
  pose = static_gestures[pose_name]
  if pose.get('right', False) is False:
    continue
  pose_points = pose['right'].to_numpy()
  
  is_a_match = True
  
  for idx, pose_point in enumerate(pose_points):
    
    x_angle = pose_point[2]
    y_angle = pose_point[3]
    z_angle = pose_point[4]
    
    base_x = static_base_gesture[idx][2]
    base_y = static_base_gesture[idx][3]
    base_z = static_base_gesture[idx][4]
    
    # Compute the error
    x_error = abs(base_x - x_angle)
    y_error = abs(base_y - y_angle)
    z_error = abs(base_z - z_angle)
    
    if(x_error > .1) or (y_error > .1) or (z_error > .1):
      is_a_match = False
  if is_a_match:
    print(f"The Pose {pose_name} matches the pose {test_pose_name} right in the middle of the video")
    # Within a threshold, say the pose is correct or not...
    # TODO: Add more test cases here, maybe add your own pictures in.....
    
  # This is the hard part... We need to fine tune the model for every static pose..
  # TODO: fine tune our error function to work for only the pose we want it to and no other poses.


The Pose one matches the pose one right in the middle of the video
